<font color=red>**Danger zone:**</font> you'll be fine-tuning a model to generate positive, negative or even toxic reviews. We'll be doing this for fun, but this is also the technique for [review bombing](https://en.wikipedia.org/wiki/Review_bomb), bot farms on social media and other less than dignified stuff. It is ultimately your decision how you apply this knowledge, but before you choose, ask yourself: is this why you chose to learn ML?


# LLMs Alignment with Reinforcement Learning from human feedback (RLHF).



In this homework, you're gonna fine-tune a language model with reinforcement learning to make it generat bad (or good) reviews.

To perform RL-based fine-tuning, we'll use a new (in this course) library called [Transformer Reinforcement Learning (TRL)](https://huggingface.co/docs/trl). TRL implements the main reinforcement learning components of RLHF: reward modeling and fine-tuning with PPO.

![img](https://huggingface.co/datasets/trl-internal-testing/example-images/resolve/main/images/TRL-readme.png)

## Stage 0: load model

To see how TRL works, we'll use it to align GPT2 on IMDB dataset to generate negative movie reviews. In fact, __it's your choice whether you want positive or negative reviews__, however I recommend you to focus on negative ones, in order to see greater effect after RLHF

But before you choose, let's take a look at the baseline model: a GPT-2 fine-tuned on generating arbitrary movie reviews.

In [1]:
import torch
import transformers
import datasets
import trl

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
main_tokenizer = transformers.AutoTokenizer.from_pretrained("lvwerra/gpt2-imdb")
main_model = transformers.AutoModelForCausalLM.from_pretrained("lvwerra/gpt2-imdb", device_map=device)

d:\miniconda3\envs\nlp-vk-05\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
d:\miniconda3\envs\nlp-vk-05\lib\site-packages\transformers\utils\generic.py:311: FutureWarning: `torch.utils._pytree._register_pytree_node` is deprecated. Please use `torch.utils._pytree.register_pytree_node` instead.
  torch.utils._pytree._register_pytree_node(
d:\miniconda3\envs\nlp-vk-05\lib\site-packages\trl\trainer\ppo_config.py:141: UserWarning: The `optimize_cuda_cache` arguement will be deprecated soon, please use `optimize_device_cache` instead.
  warnings.warn(
d:\miniconda3\envs\nlp-vk-05\lib\site-packages\huggingface_hub\file_download.py:797: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`

In [8]:
inputs = main_tokenizer("The movie", return_tensors='pt').to(device)
generated_ids = main_model.generate(**inputs, max_new_tokens=50, do_sample=True)
print("\nGenerated text:", main_tokenizer.decode(generated_ids.flatten().cpu().numpy().tolist()))

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.



Generated text: The movie is as I've seen it, if you can believe what I've heard in comments made recently, it is a complete failure. I wish that this movie had never been made. I think this was a great example to the world of acting that I


If you run this cell a couple of times, you'll see that the model generates both positive, negative and neutral reviews in some proportion. What we're gonna do next is teach the model to generate more positive (or negative) reviews.

Similarly to InstructGPT, we're gonna do that in 2 stages:
- **train a reward model** to assign higher values to positive (or negative) reviews
- fine-tune the language model to **maximize that reward using [proximal policy optimization](https://openai.com/research/openai-baselines-ppo)**



## Stage 1: train a reward model

First, we'll train a BERT-like model as our reward model. We'll generate a synthetic pairwise rankings to emulate human rankings.

__Q:__ why do I need a reward model? Can I just use a pre-trained sentiment classifier? <br> __A:__ Yes, you can - but that only works for movie reviews. But this homework will teach you how to do RLHF for any kind objective.



In [9]:
# We'll be fine-tuning a small BERT-like model for now. Please try other models for the main assignment.
reward_model = transformers.AutoModelForSequenceClassification.from_pretrained("distilbert-base-cased", device_map=device)
reward_tokenizer = transformers.AutoTokenizer.from_pretrained("distilbert-base-cased")

d:\miniconda3\envs\nlp-vk-05\lib\site-packages\huggingface_hub\file_download.py:139: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\yxtag\.cache\huggingface\hub\models--distilbert-base-cased. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)
Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-c

__Note that__ the reward model has a separate tokenizer, different from the main model. They don't need to be the same for RLHF fine-tuning.

In [32]:
from torch.utils.data import Dataset

class IMDBPairwiseDataset(Dataset):
	""" 
	A dataset of all possible pairs of chosen and rejected texts for TRL reward training format.

	This dataset is designed to facilitate the training of a reward model by providing pairs of
	texts where one is preferred (chosen) and the other is not (rejected). Each sample in the dataset
	is a dictionary containing tokenized input IDs and attention masks for both the chosen and rejected
	texts.

	Parameters:
	imdb: dataset to pairwise
	tokenizer: The tokenizer used to preprocess the texts
	accepted_label (int): The label that indicates a chosen text. Texts with this label are considered
						  preferred, while others are considered rejected.

	Methods:
	__len__(): Returns the total number of possible pairs of chosen and rejected texts.
	__getitem__(index): Returns a dictionary containing tokenized inputs for a specific pair of chosen
						and rejected texts.
	"""
	
	def __init__(self, imdb, tokenizer, accepted_label):
		super().__init__()
		self.tokenizer = tokenizer
		self.chosen_texts = imdb.filter(lambda x: x["label"] == accepted_label)["text"]
		self.rejected_texts = imdb.filter(lambda x: x["label"] != accepted_label)["text"]

		assert self.chosen_texts, f"no texts with label {accepted_label}"
		# print(f"Found {len(self.chosen_texts)} chosen and {len(self.rejected_texts)} rejected texts, {len(self)} pairs")

		self.column_names = [
			'input_ids_chosen', 'attention_mask_chosen',
			'input_ids_rejected', 'attention_mask_rejected'
		]

	def __len__(self):
		return len(self.chosen_texts) * len(self.rejected_texts)  # all pairs

	def __getitem__(self, index: int):
		choosen_text = self.chosen_texts[(index - 1) // len(self.rejected_texts)]
		rejected_text = self.rejected_texts[(index - 1) % len(self.rejected_texts)]

		chosen_inputs = self.tokenizer(choosen_text, return_tensors='pt', truncation=True)
		rejected_inputs = self.tokenizer(rejected_text, return_tensors='pt', truncation=True)
		
		return dict(
			input_ids_chosen=chosen_inputs['input_ids'][0],
			attention_mask_chosen=chosen_inputs['attention_mask'][0],
			input_ids_rejected=rejected_inputs['input_ids'][0],
			attention_mask_rejected=rejected_inputs['attention_mask'][0],
		)

In [33]:
TARGET_LABEL = 0 # negative reviews
imdb = datasets.load_dataset("imdb", split='train')
reward_data = IMDBPairwiseDataset(imdb, reward_tokenizer, accepted_label=TARGET_LABEL)

sample = reward_data[31337]
print('CHOSEN:', reward_tokenizer.decode(sample['input_ids_chosen']))
print('REJECTED:', reward_tokenizer.decode(sample['input_ids_rejected']))

CHOSEN: [CLS] If only to avoid making this type of film in the future. This film is interesting as an experiment but tells no cogent story. < br / > < br / > One might feel virtuous for sitting thru it because it touches on so many IMPORTANT issues but it does so without any discernable motive. The viewer comes away with no new perspectives ( unless one comes up with one while one's mind wanders, as it will invariably do during this pointless film ). < br / > < br / > One might better spend one's time staring out a window at a tree growing. < br / > < br / > [SEP]
REJECTED: [CLS] Well, I come from Bulgaria where it's almost impossible to have a tornado but my imagination tells me to be " very, very afraid "!!! This guy ( Devon Sawa ) has done a great job with this movie! I don't know exactly how old he was but he didn't act like a child ( WELL DONE )! Now about the tornado - it wasn't very realistic but frightens you! If you want to have a nice time in front of the telly - this is the 

We'll be using `trl.RewardTrainer` - a special case of `transformers.Trainer`.

![img](https://i.imgur.com/2JzNAPs.png)

Note that the model itself does not score pairs: it processes chosen ($y_w$) and rejected ($y_l$) samples independently. To minimize this loss, the reward model needs to score chosen sample higher than the rejected one. Note that the formula also assumes some context $x$, which is useful for seq2seq tasks. In our case of movie reviews, $x$ is empty.

In [34]:
training_args = trl.RewardConfig(  # like transformers.TrainingArguments
	output_dir="reward_model",
	per_device_train_batch_size=32,
	gradient_accumulation_steps=1,
	learning_rate=1.41e-5,
	max_steps=1_000,              # note: training may need more than 1k steps
	logging_steps=50,
	gradient_checkpointing=True,  # reduce memory usage but train ~30% slower
	gradient_checkpointing_kwargs={"use_reentrant": False},
	fp16=True,                    # disable this on CPU or on very old GPUs
	report_to='none',
	# you may add any other hyperparameters that you found useful
)

trainer = trl.RewardTrainer(
	model=reward_model,
	args=training_args,
	tokenizer=reward_tokenizer,
	train_dataset=reward_data,
	peft_config=None,  # optionally, you may tune with LoRA, prompt-tuning, etc
)

trainer.train()

d:\miniconda3\envs\nlp-vk-05\lib\site-packages\trl\trainer\reward_trainer.py:174: UserWarning: When using RewardDataCollatorWithPadding, you should set `max_length` in RewardConfig. It will be set to `512` by default, but you should do it yourself in the future.
  warnings.warn(
d:\miniconda3\envs\nlp-vk-05\lib\site-packages\trl\trainer\reward_trainer.py:191: UserWarning: When using RewardDataCollatorWithPadding, you should set `remove_unused_columns=False` in your RewardConfig we have set it for you, but you should do it yourself in the future.
  warnings.warn(
d:\miniconda3\envs\nlp-vk-05\lib\site-packages\accelerate\accelerator.py:439: FutureWarning: `torch.cuda.amp.GradScaler(args...)` is deprecated. Please use `torch.amp.GradScaler('cuda', args...)` instead.
  self.scaler = torch.cuda.amp.GradScaler(**kwargs)
  0%|          | 0/1000 [00:00<?, ?it/s]You're using a DistilBertTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than u

{'loss': 0.5332, 'learning_rate': 1.34232e-05, 'epoch': 0.0}


 10%|█         | 100/1000 [01:16<12:16,  1.22it/s]

{'loss': 0.1951, 'learning_rate': 1.2718200000000001e-05, 'epoch': 0.0}


 15%|█▌        | 150/1000 [01:50<09:40,  1.46it/s]

...

100%|██████████| 1000/1000 [11:34<00:00,  1.48it/s]

{'loss': 0.0573, 'learning_rate': 2.82e-08, 'epoch': 0.0}


100%|██████████| 1000/1000 [11:37<00:00,  1.43it/s]

{'train_runtime': 697.7943, 'train_samples_per_second': 45.859, 'train_steps_per_second': 1.433, 'train_loss': 0.11233135890960694, 'epoch': 0.0}


TrainOutput(global_step=1000, training_loss=0.11233135890960694, metrics={'train_runtime': 697.7943, 'train_samples_per_second': 45.859, 'train_steps_per_second': 1.433, 'train_loss': 0.11233135890960694, 'epoch': 0.0})

In [35]:
reward_model.gradient_checkpointing_disable()
reward_model.eval()

DistilBertForSequenceClassification(
  (distilbert): DistilBertModel(
    (embeddings): Embeddings(
      (word_embeddings): Embedding(28996, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (transformer): Transformer(
      (layer): ModuleList(
        (0-5): 6 x TransformerBlock(
          (attention): MultiHeadSelfAttention(
            (dropout): Dropout(p=0.1, inplace=False)
            (q_lin): Linear(in_features=768, out_features=768, bias=True)
            (k_lin): Linear(in_features=768, out_features=768, bias=True)
            (v_lin): Linear(in_features=768, out_features=768, bias=True)
            (out_lin): Linear(in_features=768, out_features=768, bias=True)
          )
          (sa_layer_norm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
          (ffn): FFN(
            (dropout): Dropout(p=0.1, inplace=False)
 

### Sanity-check the reward model

Let's check how our reward model performs.

__Your task__ is to measure how often does your reward model can rank a pair of (chosen and rejected) reviews correctly. Please measure this separately for train data (`imdb`) and a separate test set loaded below.

In [36]:

for sample_index in 45, 16000:
	print('TEXT:', imdb[sample_index]['text'])
	inputs = reward_tokenizer(imdb[sample_index]['text'], truncation=True, return_tensors='pt').to(device)
	with torch.no_grad():
		reward = reward_model(**inputs).logits[0, 0].item()
		print("REWARD:", reward)
	print('LABEL:', imdb[sample_index]['label'])
	print()

# note: your reward model may produce different absolute rewards.
# This is fine as long as the rewards are ordered correctly (most of the time)

TEXT: This movie sucked. It really was a waste of my life. The acting was atrocious, the plot completely implausible. Long, long story short, these people get "terrorized" by this pathetic "crazed killer", but completely fail to fight back in any manner. And this is after they take a raft on a camping trip, with no gear, and show up at a campsite that is already assembled and completely stocked with food and clothes and the daughters headphones. Additionally, after their boat goes missing, they panic that they're stuck in the woods, but then the daughters boyfriend just shows up and they apparently never consider that they could just hike out of the woods like he did to get to them. Like I said, this movie sucks. A complete joke. Don't let your girlfriend talk you into watching it.
REWARD: 5.078125
LABEL: 0

TEXT: Good: Engaging cinematic firefights, great presentation, vehicles are actually fun to drive, fairly appealing multiplayer, faithful to the movie, and the list goes on.<br /><

First of all, let's implement `compute_reward` function. Note that we use plaintext reviews because main model uses a different tokenizer from the reward model.

In [ ]:
from torch import Tensor, no_grad

def compute_reward(reward_model, reward_tokenizer, texts: list[str], device='cpu') -> Tensor:
	"""
	Compute the reward scores for a list of texts using a specified reward model and tokenizer.

	Parameters:
	reward_model: The model used to compute the reward scores
	reward_tokenizer: The tokenizer for reward_model
	texts (list[str]): A list of text strings for which the reward scores are to be computed.
	device (str, optional): The device on which the computation should be performed. Default is 'cpu'.

	Returns:
	torch.Tensor: A tensor containing the reward scores for each input text. The scores are extracted
				  from the logits of the reward model.

	Example:
	>>> compute_reward(my_reward_model, my_reward_tokenizer, ["text1", "text2"])
	tensor([ 5.1836, -4.8438], device='cpu')
	"""
	
	reward_model.to(device)
	inputs = reward_tokenizer(texts, truncation=True, padding=True, return_tensors='pt').to(device)
	
	with no_grad():
		reward = reward_model(**inputs).logits[:, 0]
	
	return reward

In [70]:
rewards = compute_reward(reward_model, reward_tokenizer, [imdb[45]['text'], imdb[16000]['text']], device=device)
print(rewards)
assert rewards[0] > rewards[1]
assert rewards[0] > 0
assert rewards[1] < 0

tensor([ 5.0742, -4.7891], device='cuda:0')


In [87]:
from tqdm.auto import tqdm

def eval_reward_model(reward_model, reward_tokenizer, test_dataset, target_label, device='cpu'):
	"""
	Evaluate the performance of a reward model by comparing reward scores for chosen and rejected reviews. 

	This function selects reviews from a test dataset based on a target label and evaluates the reward model's
	ability to assign higher scores to chosen reviews compared to rejected ones. The evaluation is performed
	in batches for efficiency.
	Note that reward scores are compared on corresponding chosen and rejected reviews: 
		chosen_reviews[0] vs rejected_reviews[0], 
		chosen_reviews[1] vs rejected_reviews[1],
		etc.

	Parameters:
	reward_model: The model used to compute the reward scores
	reward_tokenizer: The tokenizer for reward_model
	tes_dataset: test Dataset
	target_label (0 or 1): The label used to select chosen reviews. Reviews with this label are considered chosen,
				  while others are considered rejected.
	device (str, optional): The device on which the computation should be performed. Default is 'cpu'.

	Returns:
	float: The accuracy of the reward model, calculated as the proportion of times the model assigns a higher
		   reward score to the chosen review compared to the rejected review.

	Example:
	>>> accuracy = eval_reward_model(my_reward_model, my_reward_tokenizer, test_data, target_label=1)
	>>> print(f"Model accuracy: {accuracy:.2%}")
	"""

	chosen_reviews = test_dataset.filter(lambda x: x['label'] == target_label)['text']
	rejected_reviews = test_dataset.filter(lambda x: x['label'] != target_label)['text']

	if len(chosen_reviews) != len(rejected_reviews):
		min_len = min(len(chosen_reviews), len(rejected_reviews))
		chosen_reviews = chosen_reviews[:min_len]
		rejected_reviews = rejected_reviews[:min_len]

	assert len(chosen_reviews) == len(rejected_reviews)

	correct_answers_count = 0
	for chosen_review, rejected_review in tqdm(zip(chosen_reviews, rejected_reviews)):

		chosen_ids = reward_tokenizer(chosen_review, truncation=True, padding=True, return_tensors='pt').to(device)
		rejected_ids = reward_tokenizer(rejected_review, truncation=True, padding=True, return_tensors='pt').to(device)

		with torch.no_grad():
			chosen_reward = reward_model(**chosen_ids).logits[0, 0].item()
			rejected_reward = reward_model(**rejected_ids).logits[0, 0].item()
		
		if chosen_reward > rejected_reward:
			correct_answers_count += 1

	return correct_answers_count / len(chosen_reviews)

In [88]:
imdb_test = datasets.load_dataset("imdb", split='test')

test_accuracy = eval_reward_model(
	reward_model,
	reward_tokenizer,
	imdb_test,
	target_label=TARGET_LABEL,
	device=device,
)

print('test accuracy: {}'.format(test_accuracy))
assert test_accuracy > 0.94

Filter: 100%|██████████| 25000/25000 [00:00<00:00, 411892.76 examples/s]
12500it [02:48, 74.32it/s]

test accuracy: 0.96928


### Reward-guided generation (1 point)

If you did everything right, by now you should have a decent reward model. Before we use it for reinforcement learning, let's see if we can align model samples without any training.

To do so, you can use reward-guided inference: __generate N=16 samples, then select the one with the highest reward__ (according to your reward model).

For this problem, it's on you to demonstrate whether or not your code works. Find at least 5 neutral prompts such as "This movie is" (...), generate samples, rank them based on reward and show which samples get the highest reward.

Note: it is faster to generate samples in parallel, rather than sequentially, as follows:




In [89]:
prompts = ["This movie is", "The movie was", "I want to say that film", "Well, the movie was", "The movie was really"]

In [ ]:
main_tokenizer.pad_token = main_tokenizer.eos_token
main_tokenizer.padding_side = 'left'

In [ ]:
inputs = main_tokenizer(prompts, return_tensors='pt', padding=True, truncation=True).to(device)
for candidate in main_model.generate(**inputs, max_new_tokens=50, do_sample=True):
	print("Sample:", main_tokenizer.decode(candidate.flatten().cpu().numpy().tolist(), skip_special_tokens=True))

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


Sample: This movie is about a woman who accidentally kills her sister's dog. She is in love with a man with whom she dated for at first only 20 seconds. She wants revenge and she has an affair - but as a result of what happened to her, she breaks
Sample: The movie was not exactly a movie. It's more like a play. At the end of a role played by Marv Levy, I actually laughed out loud. I was not sure if he was crying or laughing in that one scene. It could have been funny
Sample: I want to say that film has been really great, though. The story was also good in certain aspects; for example, I love Cemetera, she is great in her role of the daughter of a wealthy French woman, but we have to be careful not to forget
Sample: Well, the movie was okay enough (it stars the legendary "Vladimir") but I still thought it was just plain bad! Not to mention that no one who knew who The Master of Evil was was ever introduced to it in the theaters..and I was wrong!
Sample: The movie was really good, as the

In [115]:
def generate_with_reward_guidance(
		main_model, main_tokenizer,
		reward_model, reward_tokenizer,
		N=16,
		device='cpu',
	):
	"""
	Generate text samples using a main model and select the best sample based on a reward model's guidance.

	This function generates multiple text samples from a main model, evaluates each sample using a reward model,
	and returns the sample with the highest reward score. The process is guided by the reward model to select
	the most desirable output.

	Parameters:
	main_model: The language model used to generate text samples.
	main_tokenizer: The tokenizer for main_model
	reward_model: The model used to compute reward scores for the generated samples.
	reward_tokenizer: The tokenizer for reward_model
	N (int, optional): The number of text samples to generate. Default is 16.
	device (str, optional): The device on which the computation should be performed. Default is 'cpu'.

	Returns:
	str: The generated text sample with the highest reward score.
	"""
	main_model.to(device)
	generated_ids = main_model.generate(
		max_new_tokens=100,
		do_sample=True,
		num_return_sequences=N
		)
	
	text_samples = main_tokenizer.batch_decode(
		generated_ids,
		skip_special_tokens=True
		)
	
	reward_model.to(device)
	inputs = reward_tokenizer(
		text_samples,
		truncation=True, padding=True, return_tensors='pt').to(device)
	
	with no_grad():
		reward = reward_model(**inputs).logits[:, 0]
	
	return text_samples[reward.argmax().item()]

In [116]:
generate_with_reward_guidance(
	main_model, main_tokenizer,
	reward_model, reward_tokenizer,
	device=device,
)

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
A decoder-only architecture is being used, but right-padding was detected! For correct generation results, please set `padding_side='left'` when initializing the tokenizer.


'Sigh. We all know how many things can suck in the first half: a crappy car chase, a movie that doesn\'t know what can go wrong, a plot that takes ages to figure. It\'s even worse than a "good movie". And there\'s the fact that all this movie needed is a small cameo and maybe an extra script from a couple of Hollywood stars who play big characters. The plot lines make no sense and take over 90% of the movie. It makes you wonder what'

# Stage 2: fine-tune the main model with RL


Now, we will optimize GPT2 to produce negative IMDB movie reviews using the reward model you trained above.

Unlike supervised fine-tuning, RL allows model to generate it's own sentences on each training step. Then, it calculates the reward of those specific sentences, and finally, updates the model to increase the probability of sentences with high reward.

Thus, each RLHF consists of three stages: __Rollout__, __Evaluation__ and __Update__

<div style="text-align: center">
<img src='https://huggingface.co/datasets/trl-internal-testing/example-images/resolve/main/images/gpt2_bert_training.png' width='600'>

The update stage depends on the specific RL algorithm. We'll be using Proximal Policy Optimization, or [PPO](https://arxiv.org/abs/1707.06347), similarly to what was used for InstructGPT.

Before we run those 3 stages, however, we need to create a dataset of "queries" - partial reviews in our case.

In [117]:
# Note: this code is specific to IMDB; you will need to re-write it for other tasks
imdb_for_rlhf = imdb.filter(lambda row: len(row['text']) > 200, batched=False)
imdb_for_rlhf = imdb_for_rlhf.remove_columns(['label'])
sample_length = trl.core.LengthSampler(2, 8)  # use the first 2-8 tokens as query

def select_query_and_tokenize(sample):
	query_ids = main_tokenizer.encode(sample["text"])[: sample_length()]
	sample["query"] = main_tokenizer.decode(query_ids)  # query is the only required column
	sample["input_ids"] = query_ids  # to avoid re-tokenizing later
	return sample  # we do not need the rest - it will be generated by the model

imdb_for_rlhf = imdb_for_rlhf.map(select_query_and_tokenize, batched=False)
imdb_for_rlhf.set_format(type="torch")

Map: 100%|██████████| 24895/24895 [00:17<00:00, 1446.92 examples/s]


Finally, we move to RL training. In this tutorial, we'll train LoRA adapters and not the full model.

In [118]:
import peft
peft_config = peft.LoraConfig(
	task_type=peft.TaskType.CAUSAL_LM, r=32, lora_alpha=32, lora_dropout=0.0, inference_mode=False
)

# reload main model as AutoModelForCausalLMWithValueHead - with an extra head needed for PPO
main_tokenizer = transformers.AutoTokenizer.from_pretrained("lvwerra/gpt2-imdb")
main_tokenizer.pad_token = main_tokenizer.eos_token

main_model = trl.AutoModelForCausalLMWithValueHead.from_pretrained("lvwerra/gpt2-imdb", device_map=device)
main_model = peft.get_peft_model(main_model, peft_config, adapter_name='default')
main_model.print_trainable_parameters()

d:\miniconda3\envs\nlp-vk-05\lib\site-packages\huggingface_hub\file_download.py:797: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
d:\miniconda3\envs\nlp-vk-05\lib\site-packages\accelerate\accelerator.py:439: FutureWarning: `torch.cuda.amp.GradScaler(args...)` is deprecated. Please use `torch.amp.GradScaler('cuda', args...)` instead.
  self.scaler = torch.cuda.amp.GradScaler(**kwargs)
d:\miniconda3\envs\nlp-vk-05\lib\site-packages\transformers\modeling_utils.py:488: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the

trainable params: 1,179,648 || all params: 125,620,225 || trainable%: 0.9390589771670923


Same as before, trl has a special type of trainer that minimize PPO-specific pseudo-loss. You can read more on this trainer [here](https://huggingface.co/docs/trl/main/en/ppo_trainer).

In [119]:
training_args = trl.PPOConfig(
	model_name=main_model.config._name_or_path,
	gradient_accumulation_steps=1,
	learning_rate=1.41e-5,
	batch_size=64,
	ppo_epochs=4,                 # PPO performs this many updates per training batch
)

ppo_trainer = trl.PPOTrainer(
	training_args, model=main_model.model, tokenizer=main_tokenizer,
	dataset=imdb_for_rlhf, data_collator=lambda data: dict((key, [d[key] for d in data]) for key in data[0])
)  # note: we pass main_model.model because PPOTrainer checks for one of several supported model types ...
# ... main_model.model is a model with adapters, which is supported. main_model itself is a wrapper that is not supported

In [121]:
from tqdm.auto import tqdm

max_steps = 50   # can be insufficient for some tasks - watch your learning curves
generation_kwargs = dict(
		min_length=-1, max_new_tokens=128, do_sample=True, top_k=0, top_p=1.0, pad_token_id=main_tokenizer.eos_token_id)
#                                  ^-- task-specific parameter!

average_reward = 0
gamma = 0.7

with tqdm(enumerate(ppo_trainer.dataloader), total=max_steps) as progressbar:
	# note: ppo_trainer.dataloader is just a regular dataloader of queries, no RL-specific magic :)
	for epoch, batch in progressbar:
		if epoch >= max_steps:
				break

		# Rollout stage: generate continuations from batch queries using main_model
		response_tensors = ppo_trainer.generate(batch['input_ids'], **generation_kwargs)
		# ^-- list of tensors of token ids from main model tokenizer

		# de-tokenize responses to strings (since reward model uses a different tokenizer)
		batch["response"] = [main_tokenizer.decode(response.squeeze()) for response in response_tensors]
		# note: response_tensors already contain query tokens, so we don't need to add queries manually.
		# This may not be true for other tasks: check this manually by viewing batch["response"] and batch["query"]


		# Evaluation stage - rewards for batch['response']
		rewards = compute_reward(reward_model, reward_tokenizer, batch["response"], device=device)

		# Update stage
		stats = ppo_trainer.step(batch['input_ids'], response_tensors, list(rewards.split(1)))
		stats['rewards/mean'] = rewards.mean().item()   # compute mean rewards for batch
		average_reward = gamma * average_reward + (1 - gamma) * stats['rewards/mean']

		print("-" * 30, 'STEP', epoch, '-' * 30)
		print(f'rewards/mean:\t{stats["rewards/mean"]:.9f}\t<---- average reward over this batch (higher=better, noisy)')
		print(f'rewards/moving_avg:\t{average_reward:.9f}\t<---- moving average reward (higher=better, less noisy)')
		print(f'ppo/returns/mean:\t{stats["ppo/returns/mean"]:.9f}\t<---- model-estimated average discounted reward')
		print(f'objective/kl:\t{stats["objective/kl"]:.9f}\t<---- how far we are from the original model (regularizer)')
		print()

		ppo_trainer.log_stats(stats, batch, list(rewards.split(1)))

  2%|▏         | 1/50 [00:36<29:55, 36.64s/it]

------------------------------ STEP 0 ------------------------------
rewards/mean:	0.184813499	<---- average reward over this batch (higher=better, noisy)
rewards/moving_avg:	0.055444050	<---- moving average reward (higher=better, less noisy)
ppo/returns/mean:	-0.421159863	<---- model-estimated average discounted reward
objective/kl:	0.000000000	<---- how far we are from the original model (regularizer)



  4%|▍         | 2/50 [01:13<29:38, 37.06s/it]

------------------------------ STEP 1 ------------------------------
rewards/mean:	0.408388615	<---- average reward over this batch (higher=better, noisy)
rewards/moving_avg:	0.161327419	<---- moving average reward (higher=better, less noisy)
ppo/returns/mean:	-0.383026958	<---- model-estimated average discounted reward
objective/kl:	0.069897503	<---- how far we are from the original model (regularizer)



  6%|▌         | 3/50 [01:51<29:09, 37.23s/it]

...

------------------------------ STEP 47 ------------------------------
rewards/mean:	3.096509933	<---- average reward over this batch (higher=better, noisy)
rewards/moving_avg:	3.279054684	<---- moving average reward (higher=better, less noisy)
ppo/returns/mean:	1.950516462	<---- model-estimated average discounted reward
objective/kl:	12.884355545	<---- how far we are from the original model (regularizer)



 98%|█████████▊| 49/50 [29:32<00:35, 35.55s/it]

------------------------------ STEP 48 ------------------------------
rewards/mean:	3.509265184	<---- average reward over this batch (higher=better, noisy)
rewards/moving_avg:	3.348117834	<---- moving average reward (higher=better, less noisy)
ppo/returns/mean:	2.106852770	<---- model-estimated average discounted reward
objective/kl:	15.362712860	<---- how far we are from the original model (regularizer)



100%|██████████| 50/50 [30:03<00:00, 36.07s/it]

------------------------------ STEP 49 ------------------------------
rewards/mean:	3.398410797	<---- average reward over this batch (higher=better, noisy)
rewards/moving_avg:	3.363205723	<---- moving average reward (higher=better, less noisy)
ppo/returns/mean:	1.960811734	<---- model-estimated average discounted reward
objective/kl:	15.823799133	<---- how far we are from the original model (regularizer)



In [122]:
assert average_reward > 2

And now test your PPO model:

In [124]:
inputs = [main_tokenizer.encode("The movie was", return_tensors='pt').to(device)[0] for i in range(5)]

response_tensors = ppo_trainer.generate(inputs, **generation_kwargs)
batch["response"] = [main_tokenizer.decode(response.squeeze()) for response in response_tensors]
for sample in batch["response"]:
	print('Sample: {}'.format(sample))

Sample: The movie was made by not being highschooled with the teen zombie flick, it actually made you wish you had been in it, as though it were eye candy...with a bit of self pity and misreading the art.<|endoftext|>
Sample: The movie was a remake of The Matrix - but IMDb has static images instead of static images: the anti-film take it out is junk. There is a sign published saying there is bad remake.<|endoftext|>
Sample: The movie was just called'movie', 'poster'. And the direction. o laurels make it look bad. The sales people could not have sell it. The 80's movies were just wooden from what I have seen in the past 8 years, the only movie that ever made it was called 'Do Not Miss Me'. And that movie with Hillary cyber scam exposed to people and corporations' is nothing comparison, and that movie is awful. This is the worst movie movie in garbage every time.<|endoftext|>
Sample: The movie was better than the film. It got everything superfluier than the film. For it gets to the very 